# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np

df=pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [16]:
df.id.value_counts().head()

363314    2
368962    2
568156    2
267099    2
322257    2
Name: id, dtype: int64

In [11]:
df.loc[df.id==363314]

,timestamp,id,group,action
3751,2016-11-17 15:51:56.634476,363314,control,view
3752,2016-11-17 15:52:30.776850,363314,control,click


In [25]:
clicked_ids=df[df.action=='click']['id'].unique()
viewed_ids=df[df.action=='view']['id'].unique()

clicked_and_viewed=[]
for clicker in clicked_ids:
    if clicker in viewed_ids:
        clicked_and_viewed.append(clicker)
    else:
        pass
print(f"There are {len(clicked_ids)} clickers and {len(viewed_ids)} viewers.")
print(f"There are {len(clicked_and_viewed)} users who clicked and viewed.")
print(f"There are {len(viewed_ids)-len(clicked_and_viewed)} users who viewed without clicking.")
print(f"There are {len(clicked_ids)-len(clicked_and_viewed)} users who clicked but did not view.")

There are 1860 clickers and 6328 viewers.
There are 1860 users who clicked and viewed.
There are 4468 users who viewed without clicking.
There are 0 users who clicked but did not view.


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

view     6328
click    1860
Name: action, dtype: int64

In [81]:
#Your code here
# H0: the pages were equally effective. control_clickers==experimental_clickers
# H1: the pages were not equally effective.  control_clickers!=experimental_clikers
experiment=df[df.group=='experiment']
control=df[df.group=='control']

exp_clicks=experiment[experiment.action=='click']
exp_views=experiment[experiment.action=='view']

con_clicks=control[control.action=='click']
con_views=control[control.action=='view']

exp_ratio=len(exp_clicks)/len(exp_views)
con_ratio=len(con_clicks)/len(con_views)

exp_ratio, con_ratio

(0.3097463284379172, 0.2797118847539016)

In [82]:
experiment=experiment.drop('timestamp', axis=1)

In [83]:
temp=pd.get_dummies(experiment.action)


In [92]:
exp_df=experiment.join(temp)
exp_df=exp_df.groupby('id').sum()
exp_df.head()

,click,view
id,,
182988,0,1
183136,0,1
183141,1,1
183283,0,1
183389,0,1


In [93]:
temp=pd.get_dummies(control.action)
con_df=control.join(temp)
con_df=con_df.groupby('id').sum()
con_df.head()

,click,view
id,,
182994,1,1
183089,0,1
183248,1,1
183515,0,1
183524,0,1


In [94]:
import scipy.stats as stats
stats.ttest_ind(con_df.click,exp_df.click,equal_var=False)

Ttest_indResult(statistic=-2.615440020788211, pvalue=0.008932805628674201)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [95]:
#Your code here
len(exp_df)

2996

In [98]:
expected_clicks=len(exp_df)*con_ratio
print(f"There are expected to be {round(expected_clicks,0)} clicks in the experimental group if it has the same click rate as the control group.")

There are expected to be 838.0 clicks in the experimental group if it has the same click rate as the control group.


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [105]:
#Your code here
actual_clicks=len(exp_clicks)
std=np.sqrt(len(experiment)*con_ratio*(1-con_ratio))
z=(actual_clicks-expected_clicks)/std
std,z

(28.117265621107368, 3.200282505769741)

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [110]:
#Your code here
p = stats.norm.sf(z)
p

0.000686464723597255

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**  
Both p-values are pretty small and indicate that the null hypothesis should be rejected.  The experimental page appears to be different than the control page.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.